In [ ]:
# !pip install stanza

In [ ]:
import pandas as pd
import tqdm
from collections import Counter
import stanza
import os
import numpy as np
import spacy
import tqdm
from langchain_text_splitters import RecursiveCharacterTextSplitter

/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


## Check for the agreement accuracy (pymorhy lib to check gender to avoid “вона робив”) stanza

In [ ]:
print("Downloading Ukrainian model...")
stanza.download('uk')

2025-03-04 23:32:55 INFO: Downloaded file to /Users/linndfors/stanza_resources/resources.json
2025-03-04 23:32:55 INFO: Downloading default packages for language: uk (Ukrainian) ...
2025-03-04 23:32:57 INFO: File exists: /Users/linndfors/stanza_resources/uk/default.zip
2025-03-04 23:33:02 INFO: Finished downloading models and saved to /Users/linndfors/stanza_resources


In [65]:
uk_nlp = stanza.Pipeline('uk', verbose=False, use_gpu=False)

In [16]:
def check_gender_agreement_in_sentences(sentences):
    results = {}
    for sentence in tqdm.tqdm(sentences):
        doc = uk_nlp(sentence)
        

        for sent in doc.sentences:
            for word in sent.words:
                try:
                    if word.deprel in ["nsubj", "nsubj:pass"] and "Gender" in str(word.feats):
                        if "Animacy=Inan" not in str(word.feats):
                            subj_text = word.text
                            subj_feats = word.feats
                            subj_gender = [feat.split('=')[1] for feat in subj_feats.split('|') if feat.startswith('Gender')][0]
                            subj_number = [feat.split('=')[1] for feat in subj_feats.split('|') if feat.startswith('Number')][0]

                            head_id = word.head
                            if head_id > 0:
                                verb = sent.words[head_id - 1]
                                if verb.pos == "VERB" and "Gender" in str(verb.feats):
                                    verb_text = verb.text
                                    verb_feats = verb.feats
                                    verb_gender = [feat.split('=')[1] for feat in verb_feats.split('|') if feat.startswith('Gender')][0]
                                    verb_number = [feat.split('=')[1] for feat in verb_feats.split('|') if feat.startswith('Number')][0]

                                    if subj_gender == verb_gender and subj_number == verb_number:
                                        pass
                                        # print(f"AGREE: {subj_text} ({subj_feats}) і {verb_text} ({verb_feats})")
                                    else:
                                        results[sentence] = (subj_text, verb_text)
                                        # print("\nSentence:", sentence)
                                        # print(f"XXX NOT AGREE XXX: {subj_text} ({subj_feats}) і {verb_text} ({verb_feats})")
                except Exception as e:
                    print(f"Error {e}; in the following sentence: {sentence}")
    return results

In [ ]:
results_of_agreement_test = check_gender_agreement_in_sentences(list(test_gender_swapped_df['swapped']))

  0%|          | 0/518 [00:00<?, ?it/s]

  6%|▌         | 31/518 [00:47<19:03,  2.35s/it]

Error list index out of range; in the following sentence: В наших умовах у таких «рольових коконах» продовжували дефілювати партійні ортодоксики, лектора ЦК Компартії, які своєю поведінкою дуже нагадували героїню Кіри Сазерленд (чи то героїня «Меланхолії» через двадцять п’ять років наслідує їх?..) , та ще офіцерки КҐБ, які єдині тоді мусили бути щасливі, бо їм додалося праці: в кінці травня одна така приходила вербувати й мене (цей сюжет я потім використала для схожого епізоду в біографії Віктора в «Музеї покинутих секретів»), і ми з нею близько трьох годин мужньо блукали вулицями «на свіжому повітрі», дивуючи нечастих перехожих (вона, правда, запропонувала піти до ресторану – на той час гуляти «на свіжому повітрі» вже не рекомендувалося ц


 56%|█████▌    | 289/518 [05:11<05:30,  1.44s/it]

Error list index out of range; in the following sentence: Обидві Колеснікови згадувались як членкині наглядової ради ЗАТ «Енергомережа» до того, як фірму очолила Ольга Крючкова, що останні роки представляє інтереси нардепки від БПП Ігоря Кононенка.


100%|██████████| 518/518 [08:57<00:00,  1.04s/it]


In [101]:
len(results_of_agreement_test)

19

In [66]:
results_of_agreement_dev = check_gender_agreement_in_sentences(list(dev_gender_swapped_df['swapped']))

100%|██████████| 1024/1024 [19:01<00:00,  1.11s/it]


In [102]:
len(results_of_agreement_dev)

45

In [106]:
print('agreement mistake for dev set:', "{:.4f}".format((len(results_of_agreement_dev) / len(dev_gender_swapped_df['swapped']))))

agreement mistake for dev set: 0.0439


In [105]:
print('agreement mistake for test set:', "{:.4f}".format((len(results_of_agreement_test) / len(test_gender_swapped_df['swapped']))))


agreement mistake for test set: 0.0367


In [25]:
with open('/Users/linndfors/study/diploma/uk-gender-word-mapper/common_gender_words_list.txt', 'r', encoding='utf-8') as file:
    common_gender_words_list = file.read().splitlines()

In [34]:
counter = 0

for k, v in results_of_agreement_test.items():
    v1, v2 = v
    if v1.lower() in common_gender_words_list or v2.lower() in common_gender_words_list:
        counter += 1


print(f'{counter} out of {len(results_of_agreement_test)} agreement mistakes are common gender words')

4 out of 19 agreement mistakes are common gender words


## Evaluate annotation project

In [2]:
test_swapped = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/utils_files/test_swapped.csv", index_col=0)
dev_swapped = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/utils_files/dev_swapped.csv", index_col=0)

In [3]:
df_after_annotation = pd.concat([dev_swapped, test_swapped]).reset_index()

In [7]:
(df_after_annotation['Оригінальне речення'] == df_after_annotation['Змінене речення']).value_counts()

False    1473
True       69
Name: count, dtype: int64

In [5]:
df_after_annotation['Помічник/ця'].value_counts()

Помічник/ця
Сніжанна Б              23
Олеся Д.                19
Тарас Я.                17
Оля Н.                  11
Сніжанна Б.             10
Андрій Рудницький        9
Анастасія Г              8
Крищук Катерина          7
Анна К.                  7
Вікторія Мокряк          6
Юлія М.                  6
Таміла К.                4
Королевська Ольга        4
Олександр К.             3
Дуда Луїза               3
Анастасія Г\n            3
Оля Н                    3
Ольга Ш.                 2
Королевська О.           2
Петро І.                 2
Королевська О.Д          1
Андрій Р.                1
 Сніжанна Б.             1
Мар'яна                  1
Юлія М                   1
Олеся Д,                 1
\n Андрій Рудницький     1
Name: count, dtype: int64

In [4]:
df_after_annotation.head()

,Оригінальне речення,Змінене речення,Анотації,Коректність речення,Виправлене речення,Помічник/ця
0,Виставу за п'єсою російського класика Льва Тол...,Виставу за п'єсою російського класика Льва Тол...,"{'Льва Толстого': 'PERS', 'режисера': 'JOB', '...",Містить помилки,Виставу за п'єсою російської класикині Марії Т...,Оля Н
1,Віртуозні Лесь Задніпровський – князь Абрезков...,Віртуозні Ганна Задніпровська – княгиня Абрезк...,"{'Лесь Задніпровський': 'PERS', 'князь': 'JOB'...",Містить помилки,Віртуозні Ганна Задніпровська – княгиня Абрезк...,NaN
2,"Старша в Києві , одружена з військовим , чолов...","Старша в Києві , одружена з військовою , чолов...",{'військовим': 'JOB'},Містить помилки,"Старша в Києві , одружена з військовою , жінка...",NaN
3,"Ні , не так - автор статті , позаштатний корес...","Ні , не так - авторка статті , позаштатна коре...",{'позаштатний кореспондент': 'JOB'},Правильне,NaN,NaN
4,І лектори в сірих піджаках із червоними парткв...,І лектори в сірих піджаках із червоними парткв...,{'священиків': 'JOB'},Правильне,NaN,NaN


In [60]:
print(f"Size of the dataset: {len(df_after_annotation)}\n")

print(f"Correct sentence: {len(df_after_annotation[df_after_annotation['Коректність речення']=='Правильне']) * 100 / len(df_after_annotation):.1f} %")
print(f"Uncorrect sentence: {len(df_after_annotation[df_after_annotation['Коректність речення']=='Містить помилки']) * 100 / len(df_after_annotation):.1f}%")
print(f"Difficult to determine: {len(df_after_annotation[df_after_annotation['Коректність речення']=='Важко визначити']) * 100 / len(df_after_annotation):.1f}%")

Size of the dataset: 1542

Correct sentence: 58.4 %
Uncorrect sentence: 37.5%
Difficult to determine: 3.8%


### Tokens count match for Original, GPT-generated and Corrected/Annotated Sentences

In [6]:
ner_nlp = spacy.load("uk_ner_web_trf_13class")

/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/spacy_transformers/layers/hf_shim.py:137: UserWarning: Error loading saved torch state_dict with strict=True, likely due to differences between 'transformers' versions. Attempting to load with strict=False as a fallback...

If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current 'transformers' and 'spacy-transformers' versions. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
orig_vs_gpt_match = 0
gpt_vs_ann_match = 0
orig_vs_false_gpt = 0
orig_vs_ann_match = 0
total_with_ann = 0
total = 0

for x, row in tqdm.tqdm(df_after_annotation.iterrows()):
    ann_swapped_sent = None
    orig_sent = ner_nlp(row['Оригінальне речення'])
    gpt_swapped_sent = ner_nlp(row['Змінене речення'])

    if len(orig_sent) == len(gpt_swapped_sent):
        orig_vs_gpt_match += 1
        if row['Коректність речення'] == "Містить помилки":
            orig_vs_false_gpt += 1

    if row['Коректність речення'] == "Містить помилки":
        ann_swapped_sent = ner_nlp(row['Виправлене речення'])
        total_with_ann += 1

        if len(gpt_swapped_sent) == len(ann_swapped_sent):
            gpt_vs_ann_match += 1

        if len(orig_sent) == len(ann_swapped_sent):
            orig_vs_ann_match += 1

    total += 1

1542it [19:22,  1.33it/s]


In [ ]:
print("Total sentences:", total)
print("Total with annotated correction:", total_with_ann)
print("\nMetric 1 - Orig vs GPT token count match:", orig_vs_gpt_match, f"({orig_vs_gpt_match / total:.2%})")
print("Metric 2 - GPT vs Annotated token count match:", gpt_vs_ann_match, f"({gpt_vs_ann_match / total_with_ann:.2%})")
print("Metric 3 - Orig vs Annotated token count match:", orig_vs_ann_match, f"({orig_vs_ann_match / total_with_ann:.2%})")
print("Metric 4 - Orig vs wrong GPT token count match:", orig_vs_false_gpt, f"({orig_vs_false_gpt / total_with_ann:.2%})")

Total sentences: 1542
Total with annotated correction: 579

Metric 1 - Orig vs GPT token count match: 1496 (97.02%)
Metric 2 - GPT vs Annotated token count match: 557 (96.20%)
Metric 3 - Orig vs Annotated token count match: 550 (94.99%)
Metric 4 - Orig vs wrong GPT token count match: 550 (94.99%)


## LLMs comparrison

In [ ]:
finetuned_df = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/LLM_models_comparisson/MAIN_finetuned_aya_for_the_test_parallel_dataset.csv")

In [3]:
finetuned_df

,finetuned,references,inputs,double_finetuned
0,офіцерка,офіцерка,офіцер,офіцер
1,мільярдер,мільярдер,мільярдерка,мільярдерка
2,пакистанець,пакистанець,пакистанка,пакистанка
3,Статутні внески найбільшого розміру не задекла...,Статутні внески найбільшого розміру не задекла...,Статутні внески найбільшого розміру не задекла...,Статутні внески найбільшого розміру не задекла...
4,отоларингологиня,отоларингологиня,отоларинголог,отоларинголог
...,...,...,...,...
518,"Зазначається, що це спільна власність судді та...","Зазначається, що це спільна власність судді та...","Зазначається, що це спільна власність судді та...","Зазначається, що це спільна власність судді та..."
519,"Львів», які перевірила декларації депутатів ЛМ...","Львів», які перевірила декларації депутатів ЛМ...","Львів», які перевірила декларації депутаток ЛМ...","Львів», які перевірила декларації депутаток ЛМ..."
520,- А ти що - підглядав? - питає міліціонер.,- А ти що - підглядав ? - питає міліціонер .,- А ти що - підглядав ? - питає міліціонерка .\n,- А ти що - підглядав? - питає міліціонерка.
521,Працівник розплідника Брагіна Олександр Черняє...,Працівник розплідника Брагіна Олександр Черняє...,Працівниця розплідника Брагіна Олександра Черн...,Працівниця розплідника Брагіна Валентина Черня...


In [ ]:
swapped_vs_finetuned_match = 0
orig_and_double_finetuned_match = 0
eval_total = 0

for x, row in tqdm.tqdm(finetuned_df.iterrows()):
    swapped = ner_nlp(row['references'])
    finetuned = ner_nlp(row['finetuned'])
    orig = ner_nlp(row['inputs'])
    double_finetuned = ner_nlp(row['double_finetuned'])

    if len(swapped) == len(finetuned):
        swapped_vs_finetuned_match += 1

    if len(orig) == len(double_finetuned):
        orig_and_double_finetuned_match += 1

    eval_total += 1

523it [07:44,  1.13it/s]


### Exact match for JOB entities

In [67]:
def extract_job_entities(doc):
    return list(ent.text for ent in doc.ents if ent.label_ == "JOB")

In [84]:
import tqdm

ref_finetuned_exact_match = 0
orig_double_finetuned_exact_match = 0
total_rows = len(finetuned_df)

for _, row in tqdm.tqdm(finetuned_df.iterrows(), total=total_rows):
    if row['references'] == row['finetuned']:
        ref_finetuned_exact_match += 1
    if row['inputs'] == row['double_finetuned']:
        orig_double_finetuned_exact_match += 1

100%|██████████| 523/523 [00:00<00:00, 7635.03it/s]


In [85]:
ref_finetuned_exact_match_pct = (ref_finetuned_exact_match / total_rows) * 100
orig_double_finetuned_exact_match_pct = (orig_double_finetuned_exact_match / total_rows) * 100

print(f"Exact JOB match between reference and finetuned: {ref_finetuned_exact_match_pct:.2f}%")
print(f"Exact JOB match between original and double-finetuned: {orig_double_finetuned_exact_match_pct:.2f}%")

Exact JOB match between reference and finetuned: 43.98%
Exact JOB match between original and double-finetuned: 52.01%


In [ ]:
import tqdm

ref_finetuned_match_count = 0
orig_double_finetuned_match_count = 0
total_job_ents = 0

ref_jobs_c, orig_jobs_c, finetuned_jobs_c, double_finetuned_jobs_c = 0,0,0,0

for _, row in tqdm.tqdm(finetuned_df.iterrows(), total=total_rows):
    swapped = ner_nlp(row['references'])
    finetuned = ner_nlp(row['finetuned'])
    orig = ner_nlp(row['inputs'])
    double_finetuned = ner_nlp(row['double_finetuned'])

    orig_jobs = extract_job_entities(orig)
    ref_jobs = extract_job_entities(swapped)
    finetuned_jobs = extract_job_entities(finetuned)
    double_finetuned_jobs = extract_job_entities(double_finetuned)

    ref_jobs_c += len(ref_jobs)
    orig_jobs_c += len(orig_jobs)
    finetuned_jobs_c += len(finetuned_jobs)
    double_finetuned_jobs_c += len(double_finetuned_jobs)


    len_of_job_list_swapped = min(len(ref_jobs), len(finetuned_jobs))
    
    for n in range(len_of_job_list_swapped):
        if ref_jobs[n] == finetuned_jobs[n]:
            ref_finetuned_match_count += 1

    len_of_job_list_orig = min(len(orig_jobs), len(double_finetuned_jobs))

    for n in range(len_of_job_list_orig):
        if orig_jobs[n] == double_finetuned_jobs[n]:
            orig_double_finetuned_match_count += 1

# total_job_ents = sum(ref_jobs, orig_jobs, finetuned_jobs, double_finetuned_jobs)

In [89]:
ref_finetuned_match_pct = (ref_finetuned_match_count / ref_jobs_c) * 100
orig_double_finetuned_match_pct = (orig_double_finetuned_match_count / orig_jobs_c) * 100

print(f"Exact JOB match between reference and finetuned: {ref_finetuned_match_pct:.2f}%")
print(f"Exact JOB match between original and double-finetuned: {orig_double_finetuned_match_pct:.2f}%")

Exact JOB match between reference and finetuned: 82.65%
Exact JOB match between original and double-finetuned: 87.92%


In [35]:
print("Total sentences:", eval_total)
print("\nMetric 1 - Swapped vs Finetuned token count match:", swapped_vs_finetuned_match, f"({swapped_vs_finetuned_match / eval_total:.2%})")
print("Metric 2 - Orig vs Double Finetuned token count match:", orig_and_double_finetuned_match, f"({orig_and_double_finetuned_match / eval_total:.2%})")

Total sentences: 523

Metric 1 - Swapped vs Finetuned token count match: 481 (91.97%)
Metric 2 - Orig vs Double Finetuned token count match: 487 (93.12%)


In [ ]:
orig_df = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/LLM_models_comparisson/MAIN_original_aya_for_the_test_parallel_dataset.csv")

In [57]:
orig_df.head()

,references,inputs,original,double_original
0,офіцерка,офіцер,офіцер,офіцер - жінка
1,мільярдер,мільярдерка,мільярдер,мільярдерка
2,пакистанець,пакистанка,пакистанець,пакистанка
3,Статутні внески найбільшого розміру не задекла...,Статутні внески найбільшого розміру не задекла...,Статутний внесок найбільшого розміру не задекл...,Статутний внесок найбільшого розміру не задекл...
4,отоларингологиня,отоларинголог,отоларинголог - жінка,отоларинголог - чоловік


In [ ]:
# def edit_ents(raw_output, orig_row):
#     if not orig_row.strip().endswith(".") and raw_output.endswith("."):
#         raw_output = raw_output[:-1]

#     if orig_row[0].islower():
#         return raw_output[0].lower() + raw_output[1:]
#     return raw_output

# orig_df['original'] = orig_df.apply(lambda x: edit_ents(x['original'], x['inputs']), axis=1)
# orig_df['double_original'] = orig_df.apply(lambda x: edit_ents(x['double_original'], x['original']), axis=1)
# orig_df.to_csv("/Users/linndfors/study/diploma/aya-finetuned_eval_results(comparisson)/MAIN_original_aya_for_the_test_parallel_dataset_1.csv",index=0)

In [ ]:
swapped_vs_original_match = 0
orig_and_double_original_match = 0
eval_orig_total = 0

for x, row in tqdm.tqdm(orig_df.iterrows()):
    swapped = ner_nlp(row['references'])
    original = ner_nlp(row['original'])
    orig = ner_nlp(row['inputs'])
    double_original = ner_nlp(row['double_original'])

    if len(swapped) == len(original):
        swapped_vs_original_match += 1

    if len(orig) == len(double_original):
        orig_and_double_original_match += 1

    eval_orig_total += 1

In [ ]:
print("Total sentences:", eval_orig_total)
print("\nMetric 1 - Swapped vs Original Aya token count match:", swapped_vs_original_match, f"({swapped_vs_original_match / eval_orig_total:.2%})")
print("Metric 2 - Orig vs Double Original Aya token count match:", orig_and_double_original_match, f"({orig_and_double_original_match / eval_orig_total:.2%})")

Total sentences: 523

Metric 1 - Swapped vs Original Aya token count match: 364 (69.60%)
Metric 2 - Orig vs Double Original Aya token count match: 336 (64.24%)


In [98]:
ref_orig_aya_match_count = 0
orig_double_orig_aya_match_count = 0
total_job_ents = 0

ref_jobs_c, orig_jobs_c, original_jobs_c, double_original_jobs_c = 0,0,0,0

for _, row in tqdm.tqdm(orig_df.iterrows()):
    swapped = ner_nlp(row['references'])
    original = ner_nlp(row['original'])
    orig = ner_nlp(row['inputs'])
    double_original = ner_nlp(row['double_original'])

    orig_jobs = extract_job_entities(orig)
    ref_jobs = extract_job_entities(swapped)
    original_jobs = extract_job_entities(original)
    double_original_jobs = extract_job_entities(double_original)

    ref_jobs_c += len(ref_jobs)
    orig_jobs_c += len(orig_jobs)
    original_jobs_c += len(original_jobs)
    double_original_jobs_c += len(double_original_jobs)

    len_of_job_list_swapped = min(len(ref_jobs), len(original_jobs))
    
    for n in range(len_of_job_list_swapped):
        if ref_jobs[n] == original_jobs[n]:
            ref_orig_aya_match_count += 1
        # else:
        #     print(ref_jobs[n], "-", original_jobs[n])

    len_of_job_list_orig = min(len(orig_jobs), len(double_original_jobs))

    for n in range(len_of_job_list_orig):
        if orig_jobs[n] == double_original_jobs[n]:
            orig_double_orig_aya_match_count += 1
        # else:
        #     print(orig_jobs[n], "-", double_original_jobs[n])


523it [06:56,  1.25it/s]


In [99]:
ref_original_match_pct = (ref_orig_aya_match_count / ref_jobs_c) * 100
orig_double_original_match_pct = (orig_double_orig_aya_match_count / orig_jobs_c) * 100

print(f"Exact JOB match between reference and orig Aya: {ref_original_match_pct:.2f}%")
print(f"Exact JOB match between original and double-orig Aya: {orig_double_original_match_pct:.2f}%")

Exact JOB match between reference and orig Aya: 30.36%
Exact JOB match between original and double-orig Aya: 76.09%


In [ ]:
gpt_df = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/LLM_models_comparisson/MAIN_gpt_for_the_test_parallel_dataset.csv")

In [39]:
gpt_df.head()

,references,inputs,gpt,double_gpt
0,офіцерка,офіцер,офіцерка,офіцер
1,мільярдер,мільярдерка,мільярдер,мільярдерка
2,пакистанець,пакистанка,пакистанець,пакистанка
3,Статутні внески найбільшого розміру не задекла...,Статутні внески найбільшого розміру не задекла...,Статутні внески найбільшого розміру не задекла...,Статутні внески найбільшого розміру не задекла...
4,отоларингологиня,отоларинголог,отоларингологиня,отоларинголог


In [ ]:
swapped_vs_gpt_match = 0
orig_and_double_gpt_match = 0
eval_gpt_total = 0

for x, row in tqdm.tqdm(gpt_df.iterrows()):
    swapped = ner_nlp(row['references'])
    gpt = ner_nlp(row['gpt'])
    orig = ner_nlp(row['inputs'])
    double_gpt = ner_nlp(row['double_gpt'])

    if len(swapped) == len(gpt):
        swapped_vs_gpt_match += 1

    if len(orig) == len(double_gpt):
        orig_and_double_gpt_match += 1

    eval_gpt_total += 1

0it [00:00, ?it/s]

523it [08:08,  1.07it/s]


In [55]:
print("Total sentences:", eval_gpt_total)
print("\nMetric 1 - Swapped vs GPT token count match:", swapped_vs_gpt_match, f"({swapped_vs_gpt_match / eval_gpt_total:.2%})")
print("Metric 2 - Orig vs Double GPT token count match:", orig_and_double_gpt_match, f"({orig_and_double_gpt_match / eval_gpt_total:.2%})")

Total sentences: 523

Metric 1 - Swapped vs GPT token count match: 475 (90.82%)
Metric 2 - Orig vs Double GPT token count match: 475 (90.82%)


In [ ]:
ref_gpt_match_count = 0
orig_double_gpt_match_count = 0
total_job_ents = 0

ref_jobs_c, orig_jobs_c, gpt_jobs_c, double_gpt_jobs_c = 0,0,0,0

for _, row in tqdm.tqdm(gpt_df.iterrows()):
    swapped = ner_nlp(row['references'])
    gpt = ner_nlp(row['gpt'])
    orig = ner_nlp(row['inputs'])
    double_gpt = ner_nlp(row['double_gpt'])

    orig_jobs = extract_job_entities(orig)
    ref_jobs = extract_job_entities(swapped)
    gpt_jobs = extract_job_entities(gpt)
    double_gpt_jobs = extract_job_entities(double_gpt)

    ref_jobs_c += len(ref_jobs)
    orig_jobs_c += len(orig_jobs)
    gpt_jobs_c += len(gpt_jobs)
    double_gpt_jobs_c += len(double_gpt_jobs)

    len_of_job_list_swapped = min(len(ref_jobs), len(gpt_jobs))
    
    for n in range(len_of_job_list_swapped):
        if ref_jobs[n] == gpt_jobs[n]:
            ref_gpt_match_count += 1
        else:
            print(ref_jobs[n], "-", gpt_jobs[n])

    len_of_job_list_orig = min(len(orig_jobs), len(double_gpt_jobs))

    for n in range(len_of_job_list_orig):
        if orig_jobs[n] == double_gpt_jobs[n]:
            orig_double_gpt_match_count += 1
        else:
            print(orig_jobs[n], "-", double_gpt_jobs[n])


In [ ]:
ref_gpt_match_pct = (ref_gpt_match_count / total_rows) * 100
orig_double_gpt_match_pct = (orig_double_gpt_match_count / total_rows) * 100

print(f"Exact JOB match between reference and gpt: {ref_gpt_match_pct:.2f}%")
print(f"Exact JOB match between original and double-gpt: {orig_double_gpt_match_pct:.2f}%")

### Remove PERS entities and check exact match metric

In [ ]:
import tqdm

def remove_pers_entities(doc):
    """Remove all PERS entities from the text based on their spans."""
    tokens = [token.text for token in doc if (token.ent_type_ != "PERS" and token.text != "\n")]

    return tokens

match_count_ref_finetuned = 0
match_count_orig_double = 0

total_pairs = 0

for _, row in tqdm.tqdm(finetuned_df.iterrows()):
    doc_ref = ner_nlp(row['references'])
    doc_finetuned = ner_nlp(row['finetuned'])
    doc_orig = ner_nlp(row['inputs'])
    doc_double = ner_nlp(row['double_finetuned'])

    ref_clean = remove_pers_entities(doc_ref)
    finetuned_clean = remove_pers_entities(doc_finetuned)
    orig_clean = remove_pers_entities(doc_orig)
    double_clean = remove_pers_entities(doc_double)

    total_pairs += 1

    if ref_clean == finetuned_clean:
        match_count_ref_finetuned += 1
    else:
        print(ref_clean, "--", row['references'])
        print(finetuned_clean, "--", row['finetuned'])
        print( "----\n")
        print(f"[REF ≠ FINETUNED]\n{ref_clean}\n{finetuned_clean}\n")

    if orig_clean == double_clean:
        match_count_orig_double += 1
    else:
        print(f"[ORIG ≠ DOUBLE-FINETUNED]\n{orig_clean}\n{double_clean}\n")

In [33]:
print(f"\n[STATISTICS]")
print(f"Exact matches (REF vs FINETUNED): {match_count_ref_finetuned/total_pairs}")
print(f"Exact matches (ORIG vs DOUBLE-FINETUNED): {match_count_orig_double/total_pairs}")


[STATISTICS]
Exact matches (REF vs FINETUNED): 0.6424474187380497
Exact matches (ORIG vs DOUBLE-FINETUNED): 0.739961759082218


In [ ]:
match_count_ref_gpt = 0
match_count_orig_double_gpt = 0
total_pairs = 0

for _, row in tqdm.tqdm(gpt_df.iterrows()):
    swapped = ner_nlp(row['references'])
    gpt = ner_nlp(row['gpt'])
    orig = ner_nlp(row['inputs'])
    double_gpt = ner_nlp(row['double_gpt'])

    ref_clean = remove_pers_entities(swapped)
    gpt_clean = remove_pers_entities(gpt)
    orig_clean = remove_pers_entities(orig)
    double_gpt_clean = remove_pers_entities(double_gpt)

    total_pairs += 1

    if ref_clean == gpt_clean:
        match_count_ref_gpt += 1
    else:
        print(f"[REF ≠ FINETUNED]\n{ref_clean}\n{gpt_clean}\n")

    if orig_clean == double_gpt_clean:
        match_count_orig_double_gpt += 1
    else:
        print(f"[ORIG ≠ DOUBLE-FINETUNED]\n{orig_clean}\n{double_gpt_clean}\n")

In [46]:
print(f"\n[STATISTICS]")
print(f"Exact matches (REF vs GPT): {match_count_ref_gpt/total_pairs}")
print(f"Exact matches (ORIG vs DOUBLE-GPT): {match_count_orig_double_gpt/total_pairs}")


[STATISTICS]
Exact matches (REF vs GPT): 0.6233269598470363
Exact matches (ORIG vs DOUBLE-GPT): 0.7017208413001912


In [ ]:
match_count_ref_original = 0
match_count_orig_double_original = 0
total_pairs = 0

for _, row in tqdm.tqdm(orig_df.iterrows()):
    swapped = ner_nlp(row['references'])
    original = ner_nlp(row['original'])
    orig = ner_nlp(row['inputs'])
    double_original = ner_nlp(row['double_original'])

    ref_clean = remove_pers_entities(swapped)
    original_clean = remove_pers_entities(original)
    orig_clean = remove_pers_entities(orig)
    double_original_clean = remove_pers_entities(double_original)

    total_pairs += 1

    if ref_clean == original_clean:
        match_count_ref_original += 1
    else:
        pass
        # print(f"[REF ≠ FINETUNED]\n{ref_clean}\n{original_clean}\n")

    if orig_clean == double_original_clean:
        match_count_orig_double_original += 1
    else:
        pass
        # print(f"[ORIG ≠ DOUBLE-FINETUNED]\n{orig_clean}\n{double_original_clean}\n")

0it [00:00, ?it/s]

523it [07:25,  1.17it/s]


In [60]:
print(f"\n[STATISTICS]")
print(f"Exact matches (REF vs Original Aya-101): {match_count_ref_original/total_pairs}")
print(f"Exact matches (ORIG vs DOUBLE-Aya-101): {match_count_orig_double_original/total_pairs}")


[STATISTICS]
Exact matches (REF vs Original Aya-101): 0.22562141491395793
Exact matches (ORIG vs DOUBLE-Aya-101): 0.3403441682600382


## Dictionary terms metric

In [ ]:
bruk_pairs = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/util/dataset_with_filename_and_pair_gender-swapped_words/bruk_gender_pairs_from_swapping_by_gender.csv", index_col = 0)
ng_pairs = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/util/dataset_with_filename_and_pair_gender-swapped_words/ng_gender_pairs_from_swapping_by_gender.csv", index_col = 0)

In [116]:
gender_pers_dict = pd.read_csv("/Users/linndfors/study/diploma/uk-gender-word-mapper/gender_pairs_dictionary.csv")

In [117]:
gender_pers_dict.head()

,male,female
0,абітурієнт,абітурієнтка
1,абонент,абонентка
2,абориген,аборигенка
3,абстракціоніст,абстракціоністка
4,авантюрист,авантюристка


In [143]:
with open('/Users/linndfors/study/diploma/uk-gender-word-mapper/common_gender_words_list.txt') as file:
    common_gender_words_list = [line.strip() for line in file if line.strip()]

In [156]:
in_dict = 0
not_in_dict = 0
not_in_dict_list = []

for x in ng_pairs['original']:
    if x in gender_pers_dict['male'].values or x in gender_pers_dict['female'].values or x in common_gender_words_list:
        in_dict += 1
    else:
        not_in_dict += 1
        not_in_dict_list.append(x)

print(f"Dict values: {in_dict}")
print(f"Not dict values: {not_in_dict}")
print(f"JOB for NG: dict metric: {in_dict/len(ng_pairs)}")

Dict values: 722
Not dict values: 83
JOB for NG: dict metric: 0.8968944099378882


In [157]:
in_dict = 0
not_in_dict = 0
not_in_dict_list = []

for x in bruk_pairs['original']:
    if x in gender_pers_dict['male'].values or x in gender_pers_dict['female'].values or x in common_gender_words_list:
        in_dict += 1
    else:
        not_in_dict += 1
        not_in_dict_list.append(x)
    
print(f"Dict values: {in_dict}")
print(f"Not dict values: {not_in_dict}")
print(f"JOB for BRUK: dict metric: {in_dict/len(bruk_pairs)}")

Dict values: 263
Not dict values: 56
JOB for BRUK: dict metric: 0.8244514106583072
